In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import random
import torch
import torch.optim as optim

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [ ]:
train = pd.read_csv('../input/2022-ai-w7p2/x_train.csv', encoding='CP949')
trainY = pd.read_csv('../input/2022-ai-w7p2/y_train.csv', encoding='CP949')
test = pd.read_csv('../input/2022-ai-w7p2/x_test.csv', encoding='CP949')
submit = pd.read_csv('../input/2022-ai-w7p2/sample_submission.csv', encoding='CP949')

In [ ]:
train

In [ ]:
trainX = train
testX = test

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
trainX = scaler.fit_transform(trainX)
testX = scaler.transform(testX)

In [ ]:
x_train = torch.FloatTensor(trainX).cuda()
y_train = torch.FloatTensor(trainY.to_numpy()).cuda()
x_test = torch.FloatTensor(testX).cuda()

In [ ]:
# data loader 설정
from torch.utils.data import DataLoader, TensorDataset
train_dataset = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=100)

In [ ]:
n = x_train.shape[1]
m = 512
linear1 = torch.nn.Linear(n, m, bias=True)
linear2 = torch.nn.Linear(m, m, bias=True)
linear3 = torch.nn.Linear(m, m, bias=True)
linear4 = torch.nn.Linear(m, m, bias=True)
linear5 = torch.nn.Linear(m, 1, bias=True)

relu = torch.nn.ReLU()

dropout = torch.nn.Dropout()

In [ ]:
# 자비에로 W 초기화
torch.nn.init.xavier_normal_(linear1.weight)
torch.nn.init.xavier_normal_(linear2.weight)
torch.nn.init.xavier_normal_(linear3.weight)
torch.nn.init.xavier_normal_(linear4.weight)
torch.nn.init.xavier_normal_(linear5.weight)

In [ ]:
# model 정의
model = torch.nn.Sequential(linear1, relu, dropout,
                            linear2, relu, dropout,
                            linear2, relu, dropout,
                            linear3, relu, dropout,
                            linear4, relu, dropout,
                            linear5).cuda()

In [ ]:
# 회귀는 mse
loss = torch.nn.MSELoss().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

In [ ]:
# 모델학습

total_batch = len(train_loader)
model.train()

np_epochs = 25
for epoch in range(np_epochs) :
    for x_train_batch, y_train_batch in train_loader :
        optimizer.zero_grad()
        hypothesis = model(x_train_batch)
        cost = loss(hypothesis, y_train_batch)
        cost.backward()
        optimizer.step()
        
    print(epoch, cost.item())
    
print("학습 종료")

In [ ]:
with torch.no_grad() :
    model.eval()
    
    predict = model(x_test)

In [ ]:
submit['predict'] = torch.Tensor.cpu(predict)

In [ ]:
submit.to_csv('submit12.csv', index=False)